In [1]:
import pandas as pd
import numpy as np
import string
import nltk
from sklearn.model_s# Load dataset (already uploaded)
data = pd.read_csv("fake_or_real_news.csv", usecols=['title', 'text', 'label'])

# Drop missing values
data.dropna(subset=['title', 'text', 'label'], inplace=True)

# Show first 5 rows
data.head()
election import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sowmi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Load dataset (already uploaded)
data = pd.read_csv("fake_or_real_news.csv", usecols=['title', 'text', 'label'])

# Drop missing values
data.dropna(subset=['title', 'text', 'label'], inplace=True)

# Show first 5 rows
data.head()


,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
# Initialize stemmer and stopwords
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Function for preprocessing text
def preprocess(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = text.split()
    tokens = [ps.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Apply preprocessing
data['processed_text'] = data['text'].apply(preprocess)

# Preview cleaned text
data[['text', 'processed_text']].head()


,text,processed_text
0,"Daniel Greenfield, a Shillman Journalism Fello...",daniel greenfield shillman journal fellow free...
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,googl pinterest digg linkedin reddit stumbleup...
2,U.S. Secretary of State John F. Kerry said Mon...,us secretari state john f kerri said monday st...
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",— kayde king kaydeek novemb 9 2016 lesson toni...
4,It's primary day in New York and front-runners...,primari day new york frontrunn hillari clinton...


In [4]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['processed_text']).toarray()

# Convert labels to binary: 'real' -> 1, 'fake' -> 0
y = data['label'].apply(lambda x: 1 if x.lower() == 'real' else 0)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
model = LogisticRegression()
model.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [7]:
y_pred = model.predict(X_test)

print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy Score: 0.9178386380458919

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.94      0.92       713
           1       0.93      0.89      0.91       638

    accuracy                           0.92      1351
   macro avg       0.92      0.92      0.92      1351
weighted avg       0.92      0.92      0.92      1351


Confusion Matrix:
 [[671  42]
 [ 69 569]]


In [12]:
import joblib

# Save the trained model and the exact vectorizer used
joblib.dump(model, "logistic_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
print("✅ Model and vectorizer saved.")


✅ Model and vectorizer saved.


In [11]:
def preprocess_input(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = text.split()
    tokens = [ps.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

def predict_news(news_text):
    if not news_text.strip():
        return "❗Please enter valid text."

    # Preprocess the input
    cleaned_input = preprocess_input(news_text)
    
    # Vectorize and Predict
    transformed_input = vectorizer.transform([cleaned_input]).toarray()
    prediction = model.predict(transformed_input)[0]
    
    # Show Result
    return "✅ This news article is **REAL**." if prediction == 1 else "🚫 This news article is **FAKE**."

# Input text
user_input = input("Paste the news article content here:\n")
result = predict_news(user_input)
print("\nPrediction Result:\n", result)

Paste the news article content here:
 In 2008, in the high-profile Supreme Court gun-rights case called District of Columbia v. Heller, a brief was filed from the eighth floor of the Price Daniel Sr. State Office Building in Austin, Texas, specifically from the corner office of the man who was then the stateâ€™s solicitor general, Ted Cruz.  The brief took a strong stance on the divisive question of whether the Second Amendment establishes an individual right to own guns, or just protects state and local militias. The brief argued forcefully for the first view, writing that â€œthe individual right to keep and bear armsâ€ is a â€œfundamental rightâ€ and that â€œan individual right that can be altogether abrogated is no right at all.â€ Thirty attorneys general from other states signed on.  Today, with the White House pushing new gun restrictions and Cruz's candidacy riding on next month's Iowa caucuses, it's no surprise that the Texas Republican would embrace gun rights as a defining 


Prediction Result:
 ✅ This news article is **REAL**.
